# Battle of Neighborhoods
## 1. Introduction / Business problem 

Each year number of practicing dentists in Florida grows by average by 170 [(source)](https://www.ada.org/en/science-research/health-policy-institute/data-center/supply-and-profile-of-dentists).
![](fl_dentists_chart.png)

Most of new dentists start as Associates in the established practices, but finally they want to open their own offices. 

Here comes the problem - what is the best place to open the new dental practice?

We will examine in this exercise a naïve approach of comparing population numbers in a given neighborhood to density and quality of existing dental practices.

This approach might be further expanded by comparing additional data as wealth (e.g. median household income) and commercial rent rates. 

## 2. Data